In [1]:
% matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv("./data/ds_challenge.csv")

In [5]:
data.head()

,id,city_name,signup_os,signup_channel,signup_date,bgc_date,vehicle_added_date,vehicle_make,vehicle_model,vehicle_year,first_completed_date
0,1,Strark,ios web,Paid,1/2/16,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Strark,windows,Paid,1/21/16,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Wrouver,windows,Organic,1/11/16,1/11/16,NaN,NaN,NaN,NaN,NaN
3,4,Berton,android web,Referral,1/29/16,2/3/16,2/3/16,Toyota,Corolla,2016.0,2/3/16
4,5,Strark,android web,Referral,1/10/16,1/25/16,1/26/16,Hyundai,Sonata,2016.0,NaN


In [12]:
data.columns.values

array(['id', 'city_name', 'signup_os', 'signup_channel', 'signup_date',
       'bgc_date', 'vehicle_added_date', 'vehicle_make', 'vehicle_model',
       'vehicle_year', 'first_completed_date'], dtype=object)

In [15]:
data.count()

id                      54681
city_name               54681
signup_os               47824
signup_channel          54681
signup_date             54681
bgc_date                32896
vehicle_added_date      13134
vehicle_make            13223
vehicle_model           13223
vehicle_year            13223
first_completed_date     6137
dtype: int64

In [14]:
data.count() / len(data)

id                      1.000000
city_name               1.000000
signup_os               0.874600
signup_channel          1.000000
signup_date             1.000000
bgc_date                0.601598
vehicle_added_date      0.240193
vehicle_make            0.241821
vehicle_model           0.241821
vehicle_year            0.241821
first_completed_date    0.112233
dtype: float64